In [7]:
# Install libraries within the notebook scope
sc.install_pypi_package("boto3")
sc.install_pypi_package("pandas")
sc.install_pypi_package("requests")
#sc.install_pypi_package("s3fs")
sc.install_pypi_package("fsspec")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/cc/a8/b5037dc144e458b3574c085d891b85ab2035b63ab946b5c91c23f2dfc1c6/boto3-1.16.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2b/55/9347e51769db0fe3487ed2ae5f438b3cc6aa2916e5e9d05e60a04855373e/botocore-1.19.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/25/47/22fc373440e144e2111363adaa07abb09ec1f03fbc071b6d9fc0bbf65f68/pandas-1.1.3-cp37-cp37m-manylinux1_x86_64.whl


  Using cached https://files.pythonhosted.org/packages/a5/8b/1df260f860f17cb08698170153ef7db672c497c1840dcc8613ce26a8a005/fsspec-0.8.4-py3-none-any.whl

In [9]:
import boto3
from datetime import datetime
import fsspec
import pandas as pd
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as f, types as t, Window
from pathlib import Path
import re
import requests
# import s3fs
import subprocess
import timeit
from urllib.parse import urlparse

# Removes truncation of columns, column values in Pandas
# by default
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Monkey patching the DataFrame transform method for Spark 2.4
# This is available by default in Spark 3.0
def transform(self, f):
    return f(self)
DataFrame.transform = transform

# Override the timeit template to return the command's
# return value in addition to the time
# Reference: https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module
timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

def shell_cmd(cmd):
    """
    Wrapper for running shell commands and printing the output
    Some helpful recipes:
    - List files on hdfs: shell_cmd("hdfs dfs -ls hdfs:///tmp/data/")
    - Remove files from hdfs: shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/test_pyspark")
    """
    for line in subprocess.check_output(cmd, shell=True).split(b'\n'):
        print(line)

def timer_method(cmd):
    """
    Wrapper for timeit that returns the value of a function and its runtime
    To use, pass a string of the function you wish to time
    Example: 
     run_time, result = timer_method("myfunction(arg1, arg2)")
    """
    # Setting globals = globals() enables the timeit function
    # to return the value generated by cmd
    return timeit.timeit(cmd, number=1, globals = globals())

MY_BUCKET_NAME = "data-scale-oreilly"
taxi_data_path = "s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Applying transformations to dataframes
Pyspark vs Pandas

In [10]:
def transform_taxi_ps(ps_df_taxi):
    res = (ps_df_taxi
            .select(*column_subset)
            .withColumn("tpep_pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
            .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
            .withColumn("passenger_count", f.col("passenger_count").cast(t.IntegerType()))
            .withColumn("trip_distance", f.col("trip_distance").cast(t.FloatType()))
            .withColumn("PULocationID", f.col("PULocationID").cast(t.IntegerType()))
            .withColumn("DOLocationID", f.col("DOLocationID").cast(t.IntegerType()))
            .withColumn("fare_amount", f.col("fare_amount").cast(t.FloatType()))
            .withColumn("tip_amount", f.col("tip_amount").cast(t.FloatType()))             
           )
    res.count()  # Run this to force pyspark to collect the data
    return res

def transform_taxi_pd(pd_df_taxi):
    res = pd_df_taxi[[*column_subset]].copy()
    res.tpep_pickup_datetime = pd.to_datetime(res.tpep_pickup_datetime)
    res.tpep_dropoff_datetime = pd.to_datetime(res.tpep_dropoff_datetime)
    res.passenger_count = pd.to_numeric(res.passenger_count, errors='coerce').astype('Int64')
    res.trip_distance = pd.to_numeric(res.trip_distance, errors='coerce')
    res.PULocationID = pd.to_numeric(res.PULocationID, errors='coerce').astype('Int64')
    res.DOLocationID = pd.to_numeric(res.DOLocationID, errors='coerce').astype('Int64')
    res.fare_amount = pd.to_numeric(res.fare_amount, errors='coerce')
    res.tip_amount = pd.to_numeric(res.tip_amount, errors='coerce')
    res.count()
    return res

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# pd_df_taxi = pd.read_csv(taxi_data_path, keep_default_na=False)
ps_df_taxi = spark.read.option('header', True).option('inferSchema', True).csv(taxi_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Weather data ingestion

Example of writing ingest for fixed-width data 

NOAA GHCND dataset  
https://docs.opendata.aws/noaa-ghcn-pds/readme.html  

Scroll down to 'FORMAT OF “ghcnd-stations.txt” file' for the schema of the fixed-width stations data


In [14]:
ghcnd_stations_path = "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"
stations_s3 = f"s3://{MY_BUCKET_NAME}/data/ghcnd/stations/input/ghcnd_stations.txt"
stations_local = "hdfs:///tmp/data/ghcnd/stations/input/ghcnd-stations.txt"
stations_output = "hdfs:///tmp/data/ghcnd/stations/output/ghcnd-stations.txt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Workaround reading HTTPS -> HDFS, HTTPS -> S3 -> HDFS
# Spark cant read data directly from HTTP, so copy the file to S3 and read into a dataframe from there
# Then save the file to HDFS for further processing
ingest_timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S%z")
resp = requests.get(ghcnd_stations_path)
if resp.status_code != 200:
    print("Couldn't get station data")
else:
    s3 = boto3.client('s3')
    res = s3.put_object(Body=resp.content, Bucket=MY_BUCKET_NAME, Key=f"data/ghcnd/stations/input/ghcnd_stations.txt")
    if res['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f"Unable to create ghcnd_stations.txt in s3, response {res['ResponseMetadata']['HTTPStatusCode']}")
    else:
        (spark
         .read
         .text(stations_s3)
         .write
         .format("text")
         .mode("overwrite")
         .save(stations_local))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Take a look at the stations file we just saved to HDFS
stations = spark.read.text(stations_local)
stations.show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------+
|value                                                                                |
+-------------------------------------------------------------------------------------+
|ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       |
|ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    |
|AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196|
|AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194|
|AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217|
|AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218|
|AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930|
|AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938|
|AFM00040948  34.5660   69.2120 

In [17]:
# Example of doing a substring transformation
(stations
    .withColumn("id", f.col("value").substr(0, 11))
    .drop("value")
).show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|id         |
+-----------+
|ACW00011604|
|ACW00011647|
|AE000041196|
|AEM00041194|
|AEM00041217|
|AEM00041218|
|AF000040930|
|AFM00040938|
|AFM00040948|
|AFM00040990|
+-----------+
only showing top 10 rows

### Lab - Build the ingestion for the weather stations data

Reference the fixed width schema provided under **FORMAT OF “ghcnd-stations.txt” file**   
https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt  
#### Important note - FORMAT OF is incorrect for: elevation - should be 7 char not 6

Create transformation code to convert the value column into the following schema:
* id String
* lat Float
* long Float
* elevation Float
* state String
* name String

Drop the value column, save the data in JSON format to s3://data-scale-oreilly/data/ghcnd/stations/output/section2_json 

In [18]:
def transform_stations_ps(stations_local):
    return (spark.read.text(stations_local)
     .withColumn("id", f.col("value").substr(0, 11))
     .withColumn("lat", f.col("value").substr(13, 8))
     .withColumn("long", f.col("value").substr(22, 9))
     .withColumn("elevation", f.col("value").substr(32, 7))
     .withColumn("state", f.col("value").substr(39, 2))
     .withColumn("name", f.col("value").substr(42, 30))
     .drop("value")
    )

def transform_stations_pd(stations_s3):
    return pd.read_fwf(stations_s3, 
                     [(0,10), (13, 20), (22, 30), (32, 38), (39, 40), (41, 71)],
                    names=["id", "lat", "long", "elevation", "state", "name"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
ps_run_time, stations_df = timer_method("transform_stations_ps(stations_local)")
# pd_run_time, pd_stations_df = timer_method("transform_stations_pd(stations_s3)")
# print(f"pyspark runtime: {ps_run_time} pandas runtime {pd_run_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### A key aspect of designing scalable systems is to be judicious about the data being stored and processed. 
#### The GHCND stations file contains data on stations across the US, but we are only interested in data near NYC

In [20]:
stations_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

115084

In [21]:
# Lets look at some performance tradeoffs between pyspark and pandas
# The .toPandas() function in pyspark can be convenient if you are familiar with pandas manipulations
# but this can quickly become very expensive as it collects all data on the driver to do the conversion.

def filter_ny_stations_pandas(stations_df):
    ny_stations = stations_df.filter("state == 'NY'")

    # filter down to just stations in NY in NYC. Lat of south Yonkers ~40.9124
    ny_pandas = ny_stations.toPandas()
    ny_pandas[ny_pandas.columns] = ny_pandas.apply(lambda x: x.str.strip())
    nyc_stations = ny_pandas[ny_pandas['lat'].apply(lambda x: float(x)) < 40.9124]
    res = spark.createDataFrame(nyc_stations)
    res.count()
    return res

def filter_ny_stations_pyspark(stations_df):
    print("Filtering stations to NY only")
    ny_stations = stations_df.filter("state == 'NY'")
    res = (ny_stations
            .withColumn("lat", f.col("lat").cast(t.FloatType()))
            .filter("lat < 40.9124"))
    res.count()
    return res

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
pd_run_time, pd_ny_stations = timer_method("filter_ny_stations_pandas(stations_df)")
print(f"pandas runtime {pd_run_time}")
ps_run_time, ps_ny_stations = timer_method("filter_ny_stations_pyspark(stations_df)")
print(f"pyspark runtime {ps_run_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pandas runtime 10.893973524000103
Filtering stations to NY only
pyspark runtime 0.40042098499998247

### Lab - Write two ingest functions for the stations data, one using filter_ny_stations_pandas and the other using filter_ny_stations_pyspark. What do you notice about the differences?

The functions should:
* Read the station data from local or s3
* Transform the station data into columns from the fixed width format
* Use the above filter functions
* Write the output to stations_output as json in overwrite mode

In [ ]:
def transform_station(stations):
    return (stations
     .withColumn("id", f.col("value").substr(0, 11))
     .withColumn("lat", f.col("value").substr(13, 8))
     .withColumn("long", f.col("value").substr(22, 9))
     .withColumn("elevation", f.col("value").substr(32, 7))
     .withColumn("state", f.col("value").substr(39, 2))
     .withColumn("name", f.col("value").substr(42, 30))
     .drop("value")
    )

def ingest_station_pandas():
    stations = spark.read.text(stations_local)
    (stations.transform(transform_station)
     .transform(filter_ny_stations_pandas)
     .coalesce(1)
     .write
     .mode("overwrite")
     .json(stations_output)
    )
    
def ingest_station_pyspark():
    stations = spark.read.text(stations_s3)
    (stations.transform(transform_station)
     .transform(filter_ny_stations_pyspark)
     .coalesce(1)
     .write
     .mode("overwrite")
     .json(stations_output)
    )

In [ ]:
ps_result = timer_method("ingest_station_pyspark()")
pd_result = timer_method("ingest_station_pandas()")
print(f"pandas: {pd_result} pyspark: {ps_result}")